In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from benchmark import run 

df_results = run(2, list_datasets=['mushroom'], range_nb_additional_features=[0,1,5,10])

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

sb.lineplot(data=df_results, x='nb_additional_features', y='ebm_risk_score_fit_time')
plt.show()
sb.lineplot(data=df_results, x='nb_additional_features', y='ebm_test_log_loss')

In [ ]:
df_results[['faster_risk_test_average_precision', 'ebm_test_average_precision']]

In [ ]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt


columns_fasterrisk = ['faster_risk_train_orig_log_loss',
       'faster_risk_train_orig_average_precision',
       'faster_risk_train_orig_roc_auc', 'faster_risk_train_log_loss',
       'faster_risk_train_average_precision', 'faster_risk_train_roc_auc',
       'faster_risk_calib_log_loss', 'faster_risk_calib_average_precision',
       'faster_risk_calib_roc_auc', 'faster_risk_test_log_loss',
       'faster_risk_test_average_precision', 'faster_risk_test_roc_auc']

columns_ebm = ['ebm_train_orig_log_loss', 'ebm_train_orig_average_precision',
       'ebm_train_orig_roc_auc', 'ebm_train_log_loss',
       'ebm_train_average_precision', 'ebm_train_roc_auc',
       'ebm_calib_log_loss', 'ebm_calib_average_precision',
       'ebm_calib_roc_auc', 'ebm_test_log_loss', 'ebm_test_average_precision',
       'ebm_test_roc_auc']

common_columns =  ['random_seed', 'use_calibration', 'dataset']
df_ebm = df_results[columns_ebm + common_columns]

df_ebm.columns = [c.replace('ebm_','') for c in df_ebm.columns]
df_ebm['model'] = 'scorepyo'
df_ebm['model+calib'] = df_ebm['model'] + df_ebm['use_calibration'].astype(str)

df_faster_risk = df_results[columns_fasterrisk+common_columns]
df_faster_risk.columns = [c.replace('faster_risk_','') for c in df_faster_risk.columns]
df_faster_risk['model'] = 'FasterRisk'
df_faster_risk['model+calib'] = 'FasterRisk'

df_results_reoriented = pd.concat([df_ebm, df_faster_risk], axis=0)
df_results_reoriented
for data in [
       # 'train_orig', 
       'train', 
       # 'calib', 
'test']:
    for metric in ['log_loss', 'average_precision', 'roc_auc']:
        col = data+'_'+metric
        sb.boxplot(data=df_results_reoriented, x='dataset', y=col, hue='model+calib')
        plt.show()

In [ ]:
for metric in ['average_precision', 'roc_auc']:
    df_results[f'EBM_better_{metric}'] = (df_results[f'ebm_test_{metric}'] >= df_results[f'faster_risk_test_{metric}'])
    df_results[f'EBM_better_train_{metric}'] = (df_results[f'ebm_train_{metric}'] >= df_results[f'faster_risk_train_{metric}'])

In [ ]:
for metric in ['log_loss']:
     df_results[f'EBM_better_{metric}'] = (df_results[f'ebm_test_{metric}'] <= df_results[f'faster_risk_test_{metric}'])
     df_results[f'EBM_better_train_{metric}'] = (df_results[f'ebm_train_{metric}'] <= df_results[f'faster_risk_train_{metric}'])

In [ ]:
for data in df_results['dataset'].unique():
    print(data)
    mask = df_results['dataset']==data
    for type in ['_train','']:
        for metric in ['average_precision', 'roc_auc', 'log_loss']:
            print(metric,df_results[mask][f'EBM_better{type}_{metric}'].mean())
        print()
    print()